# **Libraries**

In [1]:
#Installing Libraries Required

!pip install azure-storage-blob
!pip install snowflake-connector-python
!pip install snowflake-sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 3.3 MB/s eta 0:00:00


In [2]:
#Importing Required Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from azure.storage.blob import BlobServiceClient
import io
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import json
from snowflake.connector.pandas_tools import write_pandas
import warnings
warnings.filterwarnings('ignore')
import snowflake.connector
import dask.dataframe as dd
import pyarrow.parquet as pq
import pyarrow as pa
import os

# **Connection String Load**

In [3]:
# Load Azure and Snowflake configuration values from a JSON file for secure connection handling.

def load_config_azure_pat(config_path="config.json"):
    """Load Azure config values from JSON."""
    with open(config_path, "r", encoding="utf-8") as config_file:
        config = json.load(config_file)
    return config["AZURE_CONNECTION_STRING_PAT"], config["CONTAINER_NAME_PAT"]

def load_config_snowflake(config_path="config.json"):
    """Load Snowflake config values from JSON."""
    with open(config_path, "r", encoding="utf-8") as config_file:
        config = json.load(config_file)
    return (
        config["SNOWFLAKE_USER"],
        config["SNOWFLAKE_PASSWORD"],
        config["SNOWFLAKE_ACCOUNT"],
        config["SNOWFLAKE_WAREHOUSE"],
        config["SNOWFLAKE_DATABASE"],
        config["SNOWFLAKE_SCHEMA"]
    )

In [5]:
# Establish a connection to Snowflake using credentials and environment settings loaded from the config file.

user, password, account, warehouse, database, schema = load_config_snowflake("config.json")

conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

In [6]:
# Create a SQLAlchemy engine for Snowflake using the loaded configuration parameters.

url = URL(
    user=user,
    password=password,
    account=account,
    database=database,
    warehouse=warehouse,
    schema=schema
)

engine = create_engine(url)

# **Extraction**

In [7]:
#Extracting the data from Snowflake, all data is stored in the table AVIATION_EDGE_VIEW

query = "SELECT * FROM AVIATION_EDGE_VIEW"
cursor = conn.cursor()
cursor.execute(query)

#Converting data into a Pandas Dataframe
df = cursor.fetch_pandas_all()

cursor.close()
conn.close()

print(df.head())

  FLIGHT_TYPE  STATUS             AIRLINE_NAME AIRLINE_IATA AIRLINE_ICAO  \
0   departure  active           air tahiti nui           tn          tht   
1   departure  active          alaska airlines           as          asa   
2   departure  active               air france           af          afr   
3   departure  active  china southern airlines           cz          csn   
4   departure  active               korean air           ke          kal   

  FLIGHT_NUMBER FLIGHT_IATA_NUMBER FLIGHT_ICAO_NUMBER DEPARTURE_IATA  \
0          2311             tn2311            tht2311            sea   
1          3466             as3466            asa3466            sea   
2          8671             af8671            afr8671            sea   
3          1193             cz1193            csn1193            sea   
4          7232             ke7232            kal7232            sea   

  DEPARTURE_ICAO  ... ARRIVAL_ICAO  ARRIVAL_GATE ARRIVAL_TERMINAL  \
0           ksea  ...         kfat       

In [8]:
#Examining my dataframe

print(df.shape)
print(df.info())

(22086069, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22086069 entries, 0 to 22086068
Data columns (total 27 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   FLIGHT_TYPE              object        
 1   STATUS                   object        
 2   AIRLINE_NAME             object        
 3   AIRLINE_IATA             object        
 4   AIRLINE_ICAO             object        
 5   FLIGHT_NUMBER            object        
 6   FLIGHT_IATA_NUMBER       object        
 7   FLIGHT_ICAO_NUMBER       object        
 8   DEPARTURE_IATA           object        
 9   DEPARTURE_ICAO           object        
 10  DEPARTURE_GATE           object        
 11  DEPARTURE_DELAY_MINUTES  float64       
 12  DEPARTURE_SCHEDULED      datetime64[ns]
 13  DEPARTURE_ESTIMATED      datetime64[ns]
 14  DEPARTURE_ACTUAL         datetime64[ns]
 15  DEPARTURE_RUNWAY         datetime64[ns]
 16  ARRIVAL_IATA             object        
 17  ARRIVAL_IC

In [9]:
#Examining the sum of nulls in my dataframe

print(df.isnull().sum())

FLIGHT_TYPE                   15632
STATUS                        15632
AIRLINE_NAME                  15632
AIRLINE_IATA                  15632
AIRLINE_ICAO                  15632
FLIGHT_NUMBER                 15632
FLIGHT_IATA_NUMBER            15632
FLIGHT_ICAO_NUMBER            15632
DEPARTURE_IATA                15632
DEPARTURE_ICAO                15632
DEPARTURE_GATE              2614846
DEPARTURE_DELAY_MINUTES     1569320
DEPARTURE_SCHEDULED           15632
DEPARTURE_ESTIMATED          485735
DEPARTURE_ACTUAL            4676007
DEPARTURE_RUNWAY            4676007
ARRIVAL_IATA                  15632
ARRIVAL_ICAO                  15632
ARRIVAL_GATE                3723029
ARRIVAL_TERMINAL            9557937
ARRIVAL_BAGGAGE             9057111
ARRIVAL_SCHEDULED             15632
ARRIVAL_ESTIMATED           5149098
CODESHARE_AIRLINE          10056686
CODESHARE_FLIGHT_NUMBER    10056686
CODESHARE_FLIGHT_IATA      10056686
CODESHARE_FLIGHT_ICAO      10056686
dtype: int64


# **Data Cleaning**

In [10]:
#Dropping columns that are not relevant or useful in my analysis

df = df.drop(columns=[
    'AIRLINE_NAME',
    'AIRLINE_IATA',
    'AIRLINE_ICAO',
    'CODESHARE_FLIGHT_NUMBER',
    'CODESHARE_FLIGHT_IATA',
    'CODESHARE_FLIGHT_ICAO',
    'CODESHARE_AIRLINE',
    'ARRIVAL_BAGGAGE'
])

In [11]:
#Converting object datatype columns to strings

cols_to_string = [
    'STATUS', 'FLIGHT_IATA_NUMBER', 'FLIGHT_ICAO_NUMBER', 'DEPARTURE_IATA',
    'DEPARTURE_ICAO', 'ARRIVAL_IATA', 'ARRIVAL_ICAO', 'ARRIVAL_GATE',
    'DEPARTURE_GATE', 'ARRIVAL_TERMINAL', 'FLIGHT_TYPE'
]

df[cols_to_string] = df[cols_to_string].astype('string')

In [12]:
#Converting FLIGHT_NUMBER to an int64 datatype

df['FLIGHT_NUMBER'] = pd.to_numeric(df['FLIGHT_NUMBER']).astype('Int64')

In [13]:
#Examing the updated datatypes

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22086069 entries, 0 to 22086068
Data columns (total 19 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   FLIGHT_TYPE              string        
 1   STATUS                   string        
 2   FLIGHT_NUMBER            Int64         
 3   FLIGHT_IATA_NUMBER       string        
 4   FLIGHT_ICAO_NUMBER       string        
 5   DEPARTURE_IATA           string        
 6   DEPARTURE_ICAO           string        
 7   DEPARTURE_GATE           string        
 8   DEPARTURE_DELAY_MINUTES  float64       
 9   DEPARTURE_SCHEDULED      datetime64[ns]
 10  DEPARTURE_ESTIMATED      datetime64[ns]
 11  DEPARTURE_ACTUAL         datetime64[ns]
 12  DEPARTURE_RUNWAY         datetime64[ns]
 13  ARRIVAL_IATA             string        
 14  ARRIVAL_ICAO             string        
 15  ARRIVAL_GATE             string        
 16  ARRIVAL_TERMINAL         string        
 17  ARRIVAL_SCHEDULED        

In [14]:
#Filtering the dataframe to only contain the departure airport relevant to my analysis

airport_iata_list = ['jfk', 'atl', 'clt', 'ord', 'iah', 'ofw', 'lax', 'sea', 'mia', 'den']

df = df[df['DEPARTURE_IATA'].isin(airport_iata_list)]

In [15]:
#Examing the amount of rows and columns in my dataframe after filtering the departure airports

df.shape

(6865671, 19)

# **Data Reformatting**

In [16]:
#Creating the flight dimension

flight_df = df[['FLIGHT_NUMBER', 'FLIGHT_IATA_NUMBER', 'FLIGHT_ICAO_NUMBER', 'STATUS', 'FLIGHT_TYPE']].drop_duplicates().reset_index(drop=True)
flight_df['dim_flight_id'] = range(1000, 1000 + len(flight_df))
dim_flight = flight_df.copy()
dim_flight.columns = dim_flight.columns.str.lower()
dim_flight.head()

,flight_number,flight_iata_number,flight_icao_number,status,flight_type,dim_flight_id
0,2311,tn2311,tht2311,active,departure,1000
1,3466,as3466,asa3466,active,departure,1001
2,8671,af8671,afr8671,active,departure,1002
3,1193,cz1193,csn1193,active,departure,1003
4,7232,ke7232,kal7232,active,departure,1004


In [17]:
#Creating the gate_airport dimension

dep_gate_df = df[['DEPARTURE_GATE', 'DEPARTURE_IATA', 'DEPARTURE_ICAO']]
dep_gate_df.columns = ['gate_number', 'airport_iata', 'airport_icao']
arr_gate_df = df[['ARRIVAL_GATE', 'ARRIVAL_IATA', 'ARRIVAL_ICAO']]
arr_gate_df.columns = ['gate_number', 'airport_iata', 'airport_icao']
dim_gate = pd.concat([dep_gate_df, arr_gate_df], ignore_index=True).drop_duplicates()
dim_gate['gateairport_id'] = range(1000, 1000 + len(dim_gate))
dim_gate = dim_gate[['gateairport_id', 'gate_number', 'airport_iata', 'airport_icao']]
dim_gate.head()

,gateairport_id,gate_number,airport_iata,airport_icao
0,1000,c10f,sea,ksea
2,1001,a1,sea,ksea
8,1002,<NA>,sea,ksea
9,1003,n7,sea,ksea
15,1004,n10,sea,ksea


In [18]:
#Renaming the original dataframe to flight_fact

flight_fact = df

In [19]:
#Renaming date columns

flight_fact = flight_fact.rename(columns={
    'DEPARTURE_SCHEDULED': 'scheduled_departure_time_id',
    'DEPARTURE_ESTIMATED': 'estimated_departure_time_id',
    'ARRIVAL_SCHEDULED': 'scheduled_arrival_time_id',
    'ARRIVAL_ESTIMATED': 'estimated_arrival_time_id'
})

In [20]:
#Creating a flight duration column (in minutes) in my flight dimension


flight_fact['estimated_flight_duration'] = (
    flight_fact['estimated_arrival_time_id'] - flight_fact['estimated_departure_time_id']
).dt.total_seconds() // 60

In [21]:
#Creating a flight delay column (in minutes) based on departure estimated time and departure scheduled time

flight_fact['delay_minutes'] = (
    (flight_fact['estimated_departure_time_id'] - flight_fact['scheduled_departure_time_id'])
    .dt.total_seconds() / 60
)

In [22]:
#Examing the flight fact dimension

flight_fact.columns = flight_fact.columns.str.lower()
flight_fact.rename(columns={'estimated_flight_duration': 'estimated_flight_duration_minutes'}, inplace=True)
flight_fact.head()

,flight_type,status,flight_number,flight_iata_number,flight_icao_number,departure_iata,departure_icao,departure_gate,departure_delay_minutes,scheduled_departure_time_id,...,departure_actual,departure_runway,arrival_iata,arrival_icao,arrival_gate,arrival_terminal,scheduled_arrival_time_id,estimated_arrival_time_id,estimated_flight_duration_minutes,delay_minutes
0,departure,active,2311,tn2311,tht2311,sea,ksea,c10f,21.0,2024-11-27 14:11:00,...,2024-11-27 14:32:00,2024-11-27 14:32:00,fat,kfat,8,<NA>,2024-11-27 16:20:00,2024-11-27 16:19:00,114.0,14.0
1,departure,active,3466,as3466,asa3466,sea,ksea,c10f,21.0,2024-11-27 14:11:00,...,2024-11-27 14:32:00,2024-11-27 14:32:00,fat,kfat,8,<NA>,2024-11-27 16:20:00,2024-11-27 16:19:00,114.0,14.0
2,departure,active,8671,af8671,afr8671,sea,ksea,a1,21.0,2024-11-27 14:13:00,...,2024-11-27 14:33:00,2024-11-27 14:33:00,lax,klax,24,2,2024-11-27 17:00:00,2024-11-27 16:40:00,152.0,-5.0
3,departure,active,1193,cz1193,csn1193,sea,ksea,a1,21.0,2024-11-27 14:13:00,...,2024-11-27 14:33:00,2024-11-27 14:33:00,lax,klax,24,2,2024-11-27 17:00:00,2024-11-27 16:40:00,152.0,-5.0
4,departure,active,7232,ke7232,kal7232,sea,ksea,a1,21.0,2024-11-27 14:13:00,...,2024-11-27 14:33:00,2024-11-27 14:33:00,lax,klax,24,2,2024-11-27 17:00:00,2024-11-27 16:40:00,152.0,-5.0


# **Data Loading**

In [ ]:
#Loading the Flight Fact into Snowflake

chunksize = 10000
for i in range(0, len(flight_fact), chunksize):
    flight_fact[i:i+chunksize].to_sql('fact_flight', engine, if_exists='append', index=False, method='multi')

In [ ]:
#Loading the Flight Dimension into Snowflake

chunksize = 10000
for i in range(0, len(dim_flight), chunksize):
    dim_flight[i:i+chunksize].to_sql('dim_flight', engine, if_exists='append', index=False, method='multi')

In [ ]:
#Loading the Gate Dimension into Snowflake

chunksize = 10000
for i in range(0, len(dim_gate), chunksize):
    dim_gate[i:i+chunksize].to_sql('dim_gate', engine, if_exists='append', index=False, method='multi')